In [1]:
import polars as pl

## ERA Across Leagues

In [6]:
pitching = pl.read_csv('.\data\lahman_1871-2024_csv\Pitching.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\cv0437li\AppData\Local\Temp\ipykernel_18720\4079930422.py:1: SyntaxWarning: invalid escape sequence '\d'
  pitching = pl.read_csv('.\data\lahman_1871-2024_csv\Pitching.csv')


In [7]:
pitching.head(6)

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2
"""aardsda01""",2010,1,"""SEA""","""AL""",0,6,53,0,0,0,31,149,33,19,5,25,49,0.198,3.44,5,2,2,0,202,43,19,7,1,5


In [53]:
era_league = (
    pitching
    .filter(pl.col('yearID') >= 1946)
    .group_by('yearID', 'lgID')
    .agg(runs = pl.col('R').sum(), outs = pl.col('IPouts').sum())
    .with_columns(era = (pl.col('runs')/pl.col('outs'))*27)
    .with_columns(dh_rule = pl.when(pl.col('yearID') < 1973).then(pl.lit('No')).otherwise(
        pl.when(pl.col('yearID') >= 1973, pl.col('lgID') == pl.lit("AL")).then(pl.lit('Yes')).otherwise(
            pl.when(pl.col('yearID') >= 2022, pl.col('lgID') == pl.lit("NL")).then(pl.lit('Yes')).otherwise(
                pl.when(pl.col('yearID') == 2020).then(pl.lit('Yes')).otherwise(pl.lit('No'))
            )
        )
    )
                  )
)

era_league.head()

yearID,lgID,runs,outs,era,dh_rule
i64,str,i64,i64,f64,str
1980,"""NL""",7852,52589,4.031337,"""No"""
1971,"""AL""",7472,51874,3.889116,"""No"""
1984,"""NL""",7894,52274,4.077323,"""No"""
2001,"""NL""",12186,69223,4.753073,"""No"""
1962,"""NL""",7278,43470,4.520497,"""No"""


In [48]:
era_league.write_csv('./data/era_league.csv')

## Slugging Across Leagues

In [ ]:
batting = pl.read_csv('.\data\lahman_1871-2024_csv\Batting.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\cv0437li\AppData\Local\Temp\ipykernel_18720\4078803205.py:1: SyntaxWarning: invalid escape sequence '\d'
  batting = pl.read_csv('.\data\lahman_1871-2024_csv\Batting.csv')


In [54]:
slug = (
    batting
    .filter(pl.col('yearID') >= 1946)
    .group_by('yearID', 'lgID')
    .agg(single = pl.col('H').sum(), 
         double = pl.col('2B').sum(),
         triple = pl.col('3B').sum(),
         hr = pl.col('HR').sum(),
         at_bats = pl.col('AB').sum(),
         )
    .with_columns(slugging = (pl.col('single')+2*pl.col('double')+3*pl.col('triple')+4*pl.col('hr'))/pl.col('at_bats'))
    .with_columns(dh_rule = pl.when(pl.col('yearID') < 1973).then(pl.lit('No')).otherwise(
        pl.when(pl.col('yearID') >= 1973, pl.col('lgID') == pl.lit("AL")).then(pl.lit('Yes')).otherwise(
            pl.when(pl.col('yearID') >= 2022, pl.col('lgID') == pl.lit("NL")).then(pl.lit('Yes')).otherwise(
                pl.when(pl.col('yearID') == 2020).then(pl.lit('Yes')).otherwise(pl.lit('No'))
            )
        )
    )
                  )
)

slug.head()

yearID,lgID,single,double,triple,hr,at_bats,slugging,dh_rule
i64,str,i64,i64,i64,i64,i64,f64,str
1963,"""NL""",13434,1984,439,1215,54803,0.43025,"""No"""
2016,"""NL""",20977,4100,502,2657,82635,0.499921,"""No"""
1950,"""AL""",11474,1829,423,973,42407,0.478529,"""No"""
2023,"""AL""",20314,4130,342,2925,82324,0.501676,"""Yes"""
1972,"""AL""",14751,2260,316,1175,61712,0.403795,"""No"""


In [52]:
slug.write_csv('./data/slugging.csv')